# Crawling mautrally

from web site
http://www.consob.it/mainen/intermediares/financial_salesman/historical_register.htm. get all people information in each region. 

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

## Get all region list

In [2]:
url = 'http://www.consob.it/mainen/intermediares/financial_salesman/historical_register.htm'
html = requests.get(url).text

In [3]:
bs = BeautifulSoup(html,'lxml')

In [4]:
articles = []
for href in bs.findAll(href=re.compile(r'xsl$')):
    articles.append(href.get('href'))

## Visit all links and crawl table

example: http://www.consob.it/documenti/promotori/albo/basilicata.xml?xsl=albopromotorien.xsl

In [5]:
root = 'http://www.consob.it'

In [6]:
testurl = root+articles[1]
testhtml = requests.get(testurl).text

parse table, first loop rows, then loop column.

In [13]:
# get html table mauraly
def get_table(html):
    testbs = BeautifulSoup(html,'lxml')
    table = testbs.find('table')
    rows=list()
    for row in table.findAll("tr"):
       rows.append(row)
    result = list()
    i=0
    for line in rows:
        td = line.findAll('td')
        if re.findall('^[A-Z]',td[0].text):
            result.append((i//6,td[0].text,td[1].text.replace('\n','')))
            i+=1
    df = pd.DataFrame(result)
    df_p = df.pivot(index=0, columns=1, values=2)
    return df_p.reset_index(drop = True)

In [15]:
df_fi = pd.DataFrame()
for uri in articles[:2]:
    print (uri,'...',end='')
    url = root+uri
    html = requests.get(url).text
    df = get_table(html)
    df_fi = pd.concat([df_fi,df],ignore_index=True)
    print ('ok')

/documenti/promotori/albo/abruzzo.xml?xsl=albopromotorien.xsl ...ok
/documenti/promotori/albo/basilicata.xml?xsl=albopromotorien.xsl ...ok


In [16]:
df_fi.head()

1,Address:,Date of birth:,Intermediary:,Place of birth:,Subscrition resolution:,Surname:
0,Via Nuova Per Pianola 4 - 67042 Pianola (TE),23/05/1966,ALLIANZ BANK FINANCIAL ADVISORS SPA,L'aquila (AQ),11357 since 28/04/1998,ABBONIZIO Antonello
1,Via 403 Da Denominare 33 - 65129 Pescara (PE),23/04/1966,,Plochingen (GERMANIA),16425 since 08/04/2008,ABBONIZIO Corina
2,Via Colle Manpioppo N. 5 - 65027 Scafa (PE),26/01/1970,ALLIANZ BANK FINANCIAL ADVISORS SPA,Pescara (PE),14728 since 05/10/2004,ACETO Guerino
3,Via Marco Polo N.59 - 66054 Vasto (CH),03/03/1963,FINECOBANK BANCA FINECO SPA,Larino (CB),8997 since 17/01/1995,ADDONA Tommaso
4,Via A Di Vestea C.da Fiumicino - 64100 Teramo ...,03/03/1958,,Civitella Del Tronto (TE),5929 since 21/01/1992,AGOSTINELLI Giacomo
